# WHEN I"M DONE, I want to review what else I still need to calculate
# BASICALLY document the math for all metrics in overleaf and add to the latex wha I still need to calculate
# align the headers for this notebook and the category names in latex

In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import sys
import glob
import warnings
import random
from glob import glob 
import datetime
import itertools
import time
from multiprocessing import pool
from source.lib.helpers import *
from pandarallel import pandarallel
import ast
from functools import reduce
import yaml
import re


In [3]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pandarallel.initialize(progress_bar = False)

indir_data = Path('drive/output/derived/contributor_stats/contributor_data')
outdir_data = Path('drive/output/derived/project_outcomes')

time_period = 6#int(sys.argv[1])
df_contributor_panel = pd.read_parquet(indir_data / f"major_contributors_major_months{time_period}_window732D_samplefull.parquet")

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Organizational Characteristics

**Libraries.io API**: Use the "Repos" query, query for "created at" date. 

<span style="color:red">Not done</span>

In [4]:
df_contributor_panel#.query('comments>0 & issue_number == 0')

,repo_name,actor_id,time_period,comments,issue_comments,pr_comments,own_issue_comments,linked_pr_issue_comments,issue_number,linked_pr_issue_number,comments_10th_pct,comments_25th_pct,comments_50th_pct,comments_75th_pct,comments_90th_pct,issue_comments_10th_pct,issue_comments_25th_pct,issue_comments_50th_pct,issue_comments_75th_pct,issue_comments_90th_pct,pr_comments_10th_pct,pr_comments_25th_pct,pr_comments_50th_pct,pr_comments_75th_pct,pr_comments_90th_pct,own_issue_comments_10th_pct,own_issue_comments_25th_pct,own_issue_comments_50th_pct,own_issue_comments_75th_pct,own_issue_comments_90th_pct,linked_pr_issue_comments_10th_pct,linked_pr_issue_comments_25th_pct,linked_pr_issue_comments_50th_pct,linked_pr_issue_comments_75th_pct,linked_pr_issue_comments_90th_pct,issue_number_10th_pct,issue_number_25th_pct,issue_number_50th_pct,issue_number_75th_pct,issue_number_90th_pct,linked_pr_issue_number_10th_pct,linked_pr_issue_number_25th_pct,linked_pr_issue_number_50th_pct,linked_pr_issue_number_75th_pct,linked_pr_issue_number_90th_pct,general_comments_25th_pct,general_comments_50th_pct,general_issue_comments_25th_pct,general_issue_comments_50th_pct,general_pr_comments_25th_pct,general_pr_comments_50th_pct,general_own_issue_comments_25th_pct,general_own_issue_comments_50th_pct,general_linked_pr_issue_comments_25th_pct,general_linked_pr_issue_comments_50th_pct,general_issue_number_25th_pct,general_issue_number_50th_pct,general_linked_pr_issue_number_25th_pct,general_linked_pr_issue_number_50th_pct,pr,pr_opener,commits,commit_additions,commit_deletions,commit_changes_total,commit_files_changed_count,pr_10th_pct,pr_25th_pct,pr_50th_pct,pr_75th_pct,pr_90th_pct,pr_opener_10th_pct,pr_opener_25th_pct,pr_opener_50th_pct,pr_opener_75th_pct,pr_opener_90th_pct,commits_10th_pct,commits_25th_pct,commits_50th_pct,commits_75th_pct,commits_90th_pct,commit_additions_10th_pct,commit_additions_25th_pct,commit_additions_50th_pct,commit_additions_75th_pct,commit_additions_90th_pct,commit_deletions_10th_pct,commit_deletions_25th_pct,commit_deletions_50th_pct,commit_deletions_75th_pct,commit_deletions_90th_pct,commit_changes_total_10th_pct,commit_changes_total_25th_pct,commit_changes_total_50th_pct,commit_changes_total_75th_pct,commit_changes_total_90th_pct,commit_files_changed_count_10th_pct,commit_files_changed_count_25th_pct,commit_files_changed_count_50th_pct,commit_files_changed_count_75th_pct,commit_files_changed_count_90th_pct,general_pr_25th_pct,general_pr_50th_pct,general_pr_opener_25th_pct,general_pr_opener_50th_pct,general_commits_25th_pct,general_commits_50th_pct,general_commit_additions_25th_pct,general_commit_additions_50th_pct,general_commit_deletions_25th_pct,general_commit_deletions_50th_pct,general_commit_changes_total_25th_pct,general_commit_changes_total_50th_pct,general_commit_files_changed_count_25th_pct,general_commit_files_changed_count_50th_pct,pr_commits,pr_commit_additions,pr_commit_deletions,pr_commit_changes_total,pr_commit_files_changed_count,push_commits,push_commit_additions,push_commit_deletions,push_commit_changes_total,push_commit_files_changed_count,pr_reviews,pr_review_comments,user_type,prs_merged,issues_closed
0,4teamwork/ftw.testbrowser,7469.0,2015-01-01,5.0,0.0,5.0,3.0,5.0,3.0,3.0,1.000,1.0000,1.000,3.0000,4.200,0.00,0.0,0.0,0.00,0.00,1.000,1.0000,1.000,3.0000,4.200,0.000000,0.000000,0.000000,1.500000,2.400000,1.000000,1.000000,1.000000,3.0000,4.200,1.00,1.000,1.000000,2.0000,2.600,1.000000,1.000000,1.000000,2.000,2.60,1.0,2.00,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.00,1.0,1.0,0.0,1.00,3.0,3.0,19.0,291.0,43.0,334.0,36.0,1.000000,1.000000,1.000000,2.000000,2.600000,1.000000,1.000000,1.000000,2.000000,2.600000,1.200000,1.500000,2.00,10.5000,15.600,25.40,26.0,27.000000,159.0000,238.200,3.200000,3.500000,4.000,23.500000,35.200000,29.200000,29.500000,30.00,182.000000,273.200000,3.000000,3.000000,3.000000,19.500000,29.400000,1.0,1.0,1.0,1.0,1.000000,3.00,5.000000,35.000,2.000000,10.00,11.000000,47.00,2.000000,5.0,5.0,246.0,16.0,262.

**Forks + Stars**: Obtained from GitHub Archive, calculates number of stars/forks per time-period  
<span style="color:blue">do validation on how accurate the statistics are</span>


In [5]:
def AggregateStarsForks(indir, time_period, colname):
    df_agg = pd.concat(
        (pd.read_csv(file, usecols = ['created_at','repo_name', 'org_login']) for file in indir.glob("*.csv")),
        ignore_index=True
    )
    df_agg = ImputeTimePeriod(df_agg, time_period)
    df = df_agg.groupby(['repo_name','time_period']).size().reset_index()
    df.rename(columns = {0:colname}, inplace = True)

    org_list = df_agg['org_login'].dropna().unique().tolist()
    return org_list, df

In [6]:
indir_watch = Path("drive/output/scrape/extract_github_data/watch_data")
indir_fork = Path("drive/output/scrape/extract_github_data/fork_data")
star_orgs, df_stars = AggregateStarsForks(indir_watch, time_period, 'stars')
df_stars.rename(columns = {'stars':'stars_gained'}, inplace = True)
fork_orgs, df_fork = AggregateStarsForks(indir_fork, time_period, 'forks')
df_fork.rename(columns = {'forks':'forks_gained'}, inplace = True)
initial_org_list = list(set(star_orgs + fork_orgs))

**Creator Type (Org/Indiv)**: Whether a project was created by an organization or an individual. I obtained this by obtaining all organization names from fork, star, issues, issue comments and PR, PR reviews and PR review comment data.

In [7]:
def GetUniqueOrgs(dir_path):
    dfs = [pd.read_csv(f, usecols=["org_login"], dtype=str) for f in dir_path.glob("*.csv")]
    if not dfs:
        return set()
    combined = pd.concat(dfs, ignore_index=True)
    return combined["org_login"].dropna().unique().tolist()

In [8]:
indir_issue_raw = Path("drive/output/scrape/extract_github_data/issue_data")
indir_issue_comment_raw = Path("drive/output/scrape/extract_github_data/issue_comment_data")
indir_pull_request_raw = Path("drive/output/scrape/extract_github_data/pull_request_data")
indir_pull_request_review_comment_raw = Path("drive/output/scrape/extract_github_data/pull_request_review_comment_data")
indir_pull_request_review_raw = Path("drive/output/scrape/extract_github_data/pull_request_review_data")

all_orgs = initial_org_list
for indir in [indir_issue_raw,indir_issue_comment_raw,indir_pull_request_raw, indir_pull_request_review_comment_raw,indir_pull_request_review_raw]:
    all_orgs += GetUniqueOrgs(indir)

unique_orgs_list = list(set(all_orgs))
print("Have collected {} unique organizations".format(len(unique_orgs_list)))

Have collected 2221 unique organizations


**Project License**: Used license obtained from PyPI as of scraping date. Two cleaning changes made:
- Changed all licenses with GPL to "GNU General Public License"
- Labelled all licenses that are not MIT/BSD/Apache/GNU as "other"

<span style="color:blue">Adjust for changes to license by projects, originally downloaded on October 27, 2023 or September 29, 2024</span>


In [9]:
indir_license = Path('output/derived/collect_github_repos')
def ProcessLicenseData(indir_license):
    special_licenses = ['MIT License', 'BSD License', 'Apache Software License', 'GNU General Public License']
    df = pd.read_csv(indir_license / 'linked_pypi_github.csv', index_col=0)
    df = df[['github repository', 'license']].query('`github repository` != "Unavailable"')
    df['license'] = df['license'].parallel_apply(ast.literal_eval).apply(
        lambda x: sorted(
            set(
                ['GNU General Public License' if 'GPL' in license else license if license in special_licenses else 'Other License' 
                 for license in x if license != "Unavailable"]
            )
        )
    )
    return df
def AddIndicators(df_license):
    df = df_license.rename(columns={'github repository': 'repo_name'})
    dummies = df['license'].apply(lambda x: x if isinstance(x, list) else []).str.join('|').str.get_dummies(sep='|')
    dummies = dummies.rename(columns=lambda x: x.replace(' ', '_'))
    return pd.concat([df, dummies], axis=1)
    
df_license = ProcessLicenseData(indir_license)
df_license_indicators = AddIndicators(df_license)

**Truck Factor**
- Obtained truck factor per time-period
- Obtained "creation date" based off earliest commit date (<span style="color:blue">There are some nonsensical commit dates so I should filter for only dates after git/svn was created</span>)
- Merged each contributor to their set of emails and identified whether they had a [company domain](https://github.com/liaochris/undergrad_thesis/blob/main/data/inputs/company_domain_match_list.yaml), a .edu email or another domain. Then, I found the proportion of contributors with educational, or corporate email domains in each project and time period <span style="color:blue"> company + industry list </span>


In [10]:
def CleanTruckFactor(indir_truckfactor, time_period):
    df_truckfactor = pd.read_csv(indir_truckfactor / 'truckfactor.csv')
    df = df_truckfactor.dropna().copy()
    df['repo_name'] = (
        df['repo_name']
        .str.replace('drive/output/scrape/get_weekly_truck_factor/truckfactor_', '')
        .str.replace('.csv', '')
        .str.replace('_', '/', 1)
    )
    df = df.rename(columns={'date': 'created_at'})
    df = ImputeTimePeriod(df, time_period)
    
    df_truckfactor_period = (
        df
        .sort_values('created_at')
        .groupby(['time_period', 'repo_name'], as_index=False)
        .tail(1)
        .sort_values(['repo_name', 'time_period'])
    )
    df_truckfactor_period['authors'] = df_truckfactor_period['authors'].str.split(r'\s*\|\s*')
    df_truckfactor_period = df_truckfactor_period[['repo_name', 'time_period', 'created_at', 'week', 'year', 'truckfactor', 'authors']]

    return df_truckfactor_period


def AggregateCommitters(indir_committers):
    df_committers = pd.read_csv(indir_committers / 'committers_info.csv', index_col=0)
    df_clean = df_committers[['repo_name', 'commit_name', 'email_address', 'actor_id']].copy()
    df_clean[['commit_name', 'email_address']] = df_clean[['commit_name', 'email_address']].applymap(ast.literal_eval)
    
    df_grouped = (
        df_clean
        .explode('commit_name')
        .groupby(['commit_name', 'actor_id'])['repo_name']
        .apply(list)
        .reset_index()
    )
    
    actor_emails = (
        df_clean
        .explode('email_address')[['email_address', 'actor_id']]
        .drop_duplicates()
        .dropna()
        .groupby('actor_id')['email_address']
        .apply(list)
        .reset_index()
    )
    
    return df_grouped, actor_emails, df_clean

def MergeTruckFactorEmail(df_truckfactor_period, df_committers_grouped, actor_email_addresses):
    df = df_truckfactor_period.explode('authors')
    df = pd.merge(
        df,
        df_committers_grouped.rename(columns={'repo_name': 'repo'}),
        how='left',
        left_on='authors',
        right_on='commit_name'
    )
    
    group_cols = ['repo_name', 'time_period', 'authors']
    df['merge_count'] = df.groupby(group_cols)['truckfactor'].transform('count')
    df['repo_equal'] = (
        df['repo'].apply(lambda x: isinstance(x, list)) &
        df.apply(lambda x: x['repo_name'] in x['repo'] if isinstance(x['repo'], list) else False, axis=1)
    ).astype(int)
    df['merge_repo_match'] = df.groupby(group_cols)['repo_equal'].transform('sum')
    
    condition_na = (df['merge_count'] > 1) & ~((df['merge_repo_match'] == 1) & (df['repo_equal'] == 1))
    df_na = df[condition_na].drop(['merge_count', 'repo_equal', 'merge_repo_match', 'repo'], axis=1)
    df_na['commit_name'] = np.nan
    
    condition_valid = (df['merge_count'] == 1) | ((df['merge_count'] > 1) & (df['merge_repo_match'] == 1) & (df['repo_equal'] == 1))
    df_valid = df[condition_valid].drop(['merge_count', 'repo_equal', 'merge_repo_match', 'repo'], axis=1)
    
    df_agg = pd.concat([df_valid, df_na.drop_duplicates()], ignore_index=True)
    df_agg = df_agg.sort_values('commit_name').drop_duplicates(['repo_name', 'time_period', 'authors']).sort_values(['repo_name', 'time_period', 'authors'])
    df_agg = pd.merge(df_agg, actor_email_addresses, how='left')
    
    return df_agg

def AddCompanyAndIndustryColumns(df_truckfactor_agg):
    with open('issue/company_domain_match_list.yaml', 'r') as f:
        domains_list = yaml.load(f, Loader=yaml.FullLoader)
    
    domain_to_company = {}
    domain_to_industry = {}
    regex_mappings = []
    
    for entry in domains_list:
        company = entry.get('company')
        industry = entry.get('industry')
        for domain in entry.get('domains', []):
            domain_to_company[domain.lower()] = company
            domain_to_industry[domain.lower()] = industry

    
    df_truckfactor_agg['domain'] = df_truckfactor_agg['email_address'].apply(
        lambda x: [re.search(r'@([^@]+)$', email).group(1) for email in x  if isinstance(email, str) and re.search(r'@([^@]+)$', email)]
        if isinstance(x, list) else np.nan)
    df_truckfactor_agg['company'] = df_truckfactor_agg['domain'].apply(lambda x: [domain_to_company.get(email, np.nan) for email in x]
                                       if isinstance(x, list) else np.nan)
    df_truckfactor_agg['industry'] = df_truckfactor_agg['domain'].apply(lambda x: [domain_to_industry.get(email, np.nan) for email in x]
                                       if isinstance(x, list) else np.nan)
    
    for col in ['company','industry']:
        df_truckfactor_agg[col] = df_truckfactor_agg[col].apply(lambda x: sorted(set([email for email in x if not pd.isnull(email)])) 
                                                                if isinstance(x, list) else x)
        df_truckfactor_agg[col] = df_truckfactor_agg[col].apply(lambda x: x if isinstance(x, list) and len(x)>0 else np.nan)
        
    return df_truckfactor_agg

def AddEducationalColumns(df_truckfactor_agg):
    df_truckfactor_agg['educational'] = df_truckfactor_agg['email_address'].apply(
        lambda x: any([email.endswith(".edu") for email in x]) if isinstance(x, list) else False)
    return df_truckfactor_agg

def UniqueNonNull(s):
    return s.dropna().unique()

indir_truckfactor = Path('drive/output/scrape/get_weekly_truck_factor')
df_truckfactor_period = CleanTruckFactor(indir_truckfactor, time_period)

indir_committers = Path('drive/output/scrape/link_committers_profile')
df_committers_grouped, actor_email_addresses, df_committers_clean = AggregateCommitters(indir_committers)

df_truckfactor_agg = MergeTruckFactorEmail(df_truckfactor_period, df_committers_grouped, actor_email_addresses)
df_truckfactor_agg = AddCompanyAndIndustryColumns(df_truckfactor_agg)
df_truckfactor_agg = AddEducationalColumns(df_truckfactor_agg)
df_truckfactor_repo_summary = df_truckfactor_agg.assign(company_bool = 1-df_truckfactor_agg['company'].isna()).groupby(['repo_name','time_period','truckfactor']).agg(
    corporate_pct = ('company_bool','mean'),
    educational_pct = ('educational','mean')).reset_index()

# filter for after git was created? or after svn was created?
df_creation_date = df_truckfactor_period[['repo_name','created_at']].assign(
    created_year = df_truckfactor_period['created_at'].dt.year)\
    .query('created_year>1991')\
    .sort_values('created_at').drop_duplicates('repo_name')

**Organizational size**: \# of people making contributions, defined as opening/closing an issue, commenting on an issue, PR, reviewing a PR, merging a PR, or committing code

In [11]:
df_contributor_count = df_contributor_panel.groupby(['repo_name','time_period'])['actor_id'].count().reset_index().rename(
    {'actor_id':'contributors'}, axis = 1)

### Combine Covariates

In [12]:
def MakeBalanced(df_repo_panel):
    df_repo_panel['first_period'] = df_repo_panel.groupby('repo_name')['time_period'].transform('min')
    df_repo_panel['final_period'] = df_repo_panel.groupby('repo_name')['time_period'].transform('max')
    time_periods = df_repo_panel['time_period'].unique().tolist()
    df_balanced = df_repo_panel[['repo_name']].drop_duplicates()
    df_balanced['time_period'] = [time_periods for i in range(df_balanced.shape[0])]
    df_balanced = df_balanced.explode('time_period')
    df_repo_panel_full = pd.merge(df_balanced, df_repo_panel, how = 'left')
    df_repo_panel_full[['first_period','final_period']] = df_repo_panel_full.groupby(['repo_name'])[['first_period','final_period']].ffill()
    df_repo_panel_full = df_repo_panel_full.query('time_period >= first_period & time_period <= final_period')

    return df_repo_panel_full.drop(['first_period','final_period'], axis = 1).sort_values(['repo_name','time_period'])

In [13]:
df_repo_characteristics = reduce(lambda left, right: left.merge(right, how='outer'),
    [
        df_stars,
        df_fork,
        df_license_indicators.assign(
            license=df_license_indicators['license'].apply(lambda x: "|".join(x) if isinstance(x, list) and x else np.nan)
        ),
        df_creation_date,
        df_truckfactor_repo_summary,
        df_contributor_count
    ]
)

df_repo_characteristics = MakeBalanced(df_repo_characteristics)
df_repo_characteristics.columns = df_repo_characteristics.columns.str.lower()

def FillRepoCharacteristics(df):
    na_zero_cols = ['stars_gained', 'forks_gained', 'contributors']
    df[na_zero_cols] = df[na_zero_cols].fillna(0)
    
    fill_cols = [
        'license', 'apache_software_license', 'bsd_license',
        'gnu_general_public_license', 'mit_license', 'other_license',
        'created_at', 'created_year'
    ]
    df[fill_cols] = df.groupby('repo_name')[fill_cols].transform(lambda x: x.ffill().bfill())
    
    truck_cols = ['truckfactor', 'corporate_pct', 'educational_pct']
    df[truck_cols] = df.groupby('repo_name')[truck_cols].transform('ffill')
    return df
df_repo_characteristics = FillRepoCharacteristics(df_repo_characteristics)
df_repo_characteristics = df_repo_characteristics.query('time_period>="2015-01-01"')

In [14]:
df_repo_characteristics

,repo_name,time_period,stars_gained,forks_gained,license,apache_software_license,bsd_license,gnu_general_public_license,mit_license,other_license,created_at,created_year,truckfactor,corporate_pct,educational_pct,contributors
0,007gzs/django_restframework_apiview,2017-01-01,2.0,0.0,GNU General Public License,0.0,0.0,1.0,0.0,0.0,2017-06-27 05:05:52,2017.0,1.0,0.0,0.0,2.0
1,007gzs/django_restframework_apiview,2017-07-01,0.0,0.0,GNU General Public License,0.0,0.0,1.0,0.0,0.0,2017-06-27 05:05:52,2017.0,1.0,0.0,0.0,1.0
2,007gzs/django_restframework_apiview,2018-01-01,18.0,0.0,GNU General Public License,0.0,0.0,1.0,0.0,0.0,2017-06-27 05:05:52,2017.0,1.0,0.0,0.0,1.0
3,007gzs/django_restframework_apiview,2018-07-01,1.0,0.0,GNU General Public License,0.0,0.0,1.0,0.0,0.0,2017-06-27 05:05:52,2017.0,1.0,0.0,0.0,0.0
4,007gzs/django_restframework_apiview,2019-01-01,1.0,1.0,GNU General Public License,0.0,0.0,1.0,0.0,0.0,2017-06-27 05:05:52,2017.0,1.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608201,zzzsochi/trans,2020-01-01,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,2011-11-29 07:08:52,2011.0,1.0,0.0,0.0,0.0
608202,zzzsochi/trans,2020-07-01,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,2011-11-29 07:08:52,2011.0,1.0,0.0,0.0,0.0
608203,zzzsochi/trans,2021-01-01,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2011-11-29 07:08:52,2011.0,1.0,0.0,0.0,0.0
608210,zzzsochi/trans,2021-07-01,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2011-11-29 07:08:52,2011.0,1.0,0.0,0.0,0.0


## Individual Characteristics

**Email domain**: Whether in a project-contributor-time period, an individual made a commit with an educational or corporate email domain

In [15]:
df_contributor_email_category = df_truckfactor_agg.assign(company_bool = ~df_truckfactor_agg['company'].isna())\
    .groupby(['repo_name','actor_id','time_period'])\
    .agg(contributor_email_educational = ('educational','max'), 
         contributor_email_corporate = ('company_bool','max')).reset_index()
df_contributor_email_category[['contributor_email_educational','contributor_email_corporate']] = df_contributor_email_category[['contributor_email_educational','contributor_email_corporate']].astype(int)

1) Max Rank for each individual in each period as a continuous rank variable
2) Rank indicator
3) % for each rank that they handled

df_contributor_panel

In [16]:
def ContributorRankCharacteristics(df_contributor_panel):
    val_cols = [col for col in df_contributor_panel.columns if 'pct' in col]
    df = df_contributor_panel.drop(columns=val_cols).rename(columns={'issue_number': 'issues_opened', 'pr': 'prs_opened'}).reset_index(drop=True)
    
    df['problem_identification_sum'] = df['issues_opened']
    df['problem_identification'] = df['problem_identification_sum'] > 0
    
    #df['problem_discussion_sum'] = df[['comments', 'own_pr_comments', 'helping_pr_comments', 'pr_reviews', 'pr_review_comments']].sum(axis=1)
    df['problem_discussion_sum'] = df[['comments', 'pr_comments', 'pr_reviews', 'pr_review_comments']].sum(axis=1)
    df['problem_discussion'] = df['problem_discussion_sum'] > 0
    
    df['coding_sum'] = df[['prs_opened', 'commits']].sum(axis=1)
    df['coding'] = df['coding_sum'] > 0
    
    df['problem_approval_sum'] = df[['prs_merged', 'issues_closed']].sum(axis=1)
    df['problem_approval'] = df['problem_approval_sum'] > 0
    
    df['max_rank'] = np.select(
        [
            df['problem_approval'],
            df['coding'],
            df['problem_discussion']
        ],
        [4, 3, 2],
        default=1
    )
    agg_cols = ['problem_identification', 'problem_discussion', 'coding', 'problem_approval']
    sum_cols = [f'{c}_sum' for c in agg_cols]
    grouped_sums = df.groupby(['repo_name', 'time_period'])[sum_cols].transform('sum')
    for col in sum_cols:
        df[f'{col.replace("_sum","")}_share'] = df[col] / grouped_sums[col]
    df['total_share'] = df[sum_cols].sum(axis = 1) / grouped_sums.sum(axis = 1)
    rank_activity_cols = [
        'issues_opened', 'problem_identification_sum', 'comments', 
        #'own_pr_comments', 'helping_pr_comments', 
        'pr_comments',
        'pr_reviews', 'pr_review_comments', 'problem_discussion_sum',
        'prs_opened', 'commits', 'coding_sum', 'prs_merged', 'issues_closed',
        'problem_approval_sum'
    ]
    final_cols = ['actor_id', 'repo_name', 'time_period', 'max_rank'] + agg_cols + [f'{c}_share' for c in agg_cols] + rank_activity_cols + ['total_share']
    return df[final_cols]

df_contributor_panel_rank = ContributorRankCharacteristics(df_contributor_panel)

**Truckfactor Member**: Indicator variable for whether in that time period they were part of the truckfactor

In [17]:
df_truckfactor_dates = df_truckfactor_agg[['repo_name','time_period','actor_id']].assign(
    truckfactor_member = 1)

**% of LOC authored**: Using the git blame %, identify the % of all code that the contributor had authored in the codebase in that project-time period 

<span style="color:red">To Do</span>

- **Work Share**: For each project-contributor-time period, in **(1) discussions** or **(2) pull requests**, what % of the work did they do? <span style="color:blue">Note that for the three below, one problem can reappear in multiple time periods if work is done on the same problem in multiple periods. I may want to experiment with alternatives, such as counting it all in the original period</span>

  - Share, average: $\frac{1}{|P_i|} \sum_{p \in P_i}^n S_{p, i}$ where $P_i$ contains all problems $p$ (comments or pull requests) contributor $i$ was involved in and $S_{p, i}$ represents the share of work contributor $i$ did in problem $p$
  - Share, average weighted by work: $\sum_{p \in P_i}^n S_{p, i} \cdot \frac{C_{p,i}}{\sum_{p \in P_i} C_{p,i}}$  where $C_{p, i}$ represents the quantity of work contributor $i$ did in problem $p$. Now, instead of weighting each problem equally, we weight each problem $p$ by how much work contributor $i$ did in problem $p$
- **Individual HHI**: For each project-contributor-time period, in **(1) discussions** or **(2) pull requests** that they were involved in, what was the HHI? This tells us how balanced the work was when they were involved
  - Share, average: $\frac{1}{|P_i|} \sum_{p \in P_i}^n HHI_{p}$ where $P_i$ contains all problems $p$ (comments or pull requests) contributor $i$ was involved in and $HHI_{p}$ represents the HHI of problem $p$
  - Share, average weighted by work: $\sum_{p \in P_i}^n HHI_{p} \cdot \frac{C_{p,i}}{\sum_{p \in P_i} C_{p,i}}$ where $C_{p, i}$ represents the quantity of work contributor $i$ did in problem $p$. Now, instead of weighting each problem equally, we weight each problem $p$ by how much work contributor $i$ did in problem $p$
- **Cooperation**: % of **(1) discussions** or **(2) pull requests** that they were involved in that had another contributor besides them


In [18]:
indir_export = Path('drive/output/derived/data_export')
df_issue = pd.read_parquet(indir_export / 'df_issue.parquet')
df_issue = ImputeTimePeriod(df_issue, time_period)

df_pr = pd.read_parquet(indir_export / 'df_pr.parquet')
df_pr = ImputeTimePeriod(df_pr, time_period)

df_pr_commits = pd.read_parquet(indir_export / 'df_pr_commits.parquet')

def IndivLevelComments(df_issue, df_pr):
    df_issue_comments = df_issue.query('type == "IssueCommentEvent"').sort_values('created_at').drop_duplicates('issue_comment_id')
    df_pr_rc = df_pr.query('type == "PullRequestReviewCommentEvent"')
    df_pr_rc['issue_number'] =  df_pr_rc['pr_number']
    df_pr_rc = df_pr_rc.sort_values('created_at').drop_duplicates('pr_review_comment_body')
    
    df_comments_actor = pd.concat([df_pr_rc[['repo_name','actor_id','time_period','issue_number','created_at']],
                                   df_issue_comments[['repo_name','actor_id','time_period','issue_number','created_at']]])\
        .groupby(['repo_name','actor_id','time_period','issue_number'])['created_at'].count()\
        .reset_index().rename({'created_at':'comments'}, axis = 1)
    return df_comments_actor

def IndivLevelCommits(df_pr_commits):
    df_pr_commits['created_at'] = pd.to_datetime(df_pr_commits['commit time'], unit = 's')
    df_pr_commits = df_pr_commits.query('~created_at.isna()')
    df_pr_commits = ImputeTimePeriod(df_pr_commits, time_period)
    df_committers_ids = df_committers_clean.explode('commit_name').explode('email_address')[['commit_name','email_address','actor_id']].drop_duplicates()
    df_pr_commits_id = pd.merge(df_pr_commits, df_committers_ids, how = 'left', left_on = ['commit author name','commit author email'], right_on = ['commit_name','email_address'])
    df_pr_commits_id['commits'] = 1
    df_pr_commits_id['commits_100'] = (df_pr_commits_id['commit files changed count']<100).astype(int)

    #### NOT GOING TO WEIGHT BY ADDITIONS, DELETIONS FOR NOW BUT GOOD TO HAVE
    df_pr_commits_actor = df_pr_commits_id.query('~actor_id.isna()').groupby(['repo_name','time_period','actor_id','pr_number']).agg(
        commits_count = ('commits','sum'),
        commits_100_count = ('commits_100','sum'),
        commits_add_sum = ('commit additions','sum'),
        commits_del_sum = ('commit deletions','sum'),
        commits_change_sum = ('commit changes total','sum'),
        commits_files_changed_count = ('commit files changed count','sum')).reset_index()
        
    return df_pr_commits_actor

def AggregateHHICount(df, id_col, count_col):
    indiv_total_col = f'indiv_total_{count_col}'
    discussions_col = f'{count_col}_discussions_involved'
    total_col = f'total_{count_col}'
    share_col = f'{count_col}_share'
    hhi_col = f'{count_col}_hhi'
    indiv_col = f'{count_col}_indiv'
    wm = lambda x: np.average(x, weights=df.loc[x.index, count_col])

    df[discussions_col] = df.groupby(['repo_name','time_period','actor_id'])[count_col].transform('count')
    df[indiv_total_col] = df.groupby(['repo_name','time_period','actor_id'])[count_col].transform('sum')
    df[total_col] = df.groupby(['repo_name','time_period',id_col])[count_col].transform('sum')
    df[share_col] = df[count_col]/df[total_col]
    df[hhi_col] = df.assign(share_sq = lambda x: x[share_col]**2)\
        .groupby(['repo_name','time_period',id_col])['share_sq'].transform('sum')
    df[indiv_col] = (df[share_col]<1).astype(int)
    df[f'{share_col}_wt'] = df[share_col] * df[count_col]/df[indiv_total_col]
    df[f'{hhi_col}_wt'] = df[hhi_col] * df[count_col]/df[indiv_total_col]
    
    df_grouped = df.groupby(['repo_name','actor_id','time_period',indiv_total_col,discussions_col]).agg(
        **{f'{share_col}_avg': (share_col, 'mean'),
           f'{share_col}_avg_wt': (f'{share_col}_wt', 'sum'),
           f'{hhi_col}_avg': (hhi_col, 'mean'),
           f'{hhi_col}_avg_wt': (f'{hhi_col}_wt', 'sum'),
           f'pct_cooperation_{count_col}': (indiv_col, 'mean')}).reset_index()
    
    return df_grouped

In [19]:
df_comments_actor = IndivLevelComments(df_issue, df_pr)
df_pr_commits_actor = IndivLevelCommits(df_pr_commits)

df_comments_grouped = AggregateHHICount(df_comments_actor, 'issue_number', 'comments')

df_pr_commits_grouped = AggregateHHICount(df_pr_commits_actor, 'pr_number', 'commits_count')
df_pr_commits_grouped = df_pr_commits_grouped.rename({'commits_count_discussions_involved':'pr_discussions_involved'}, axis = 1)

df_pr_commits_100_grouped = AggregateHHICount(df_pr_commits_actor, 'pr_number', 'commits_100_count')
df_pr_commits_100_grouped = df_pr_commits_100_grouped.rename({'commits_100_count_discussions_involved':'100_pr_discussions_involved'}, axis = 1)

### Combine Covariates

In [20]:
def MakeBalancedContributor(df_contributor_panel):
    df_contributor_panel['first_period'] = df_contributor_panel.groupby(['repo_name','actor_id'])['time_period'].transform('min')
    df_contributor_panel['final_period'] = df_contributor_panel.groupby(['repo_name','actor_id'])['time_period'].transform('max')
    time_periods = df_contributor_panel['time_period'].unique().tolist()
    df_balanced = df_contributor_panel[['repo_name','actor_id','first_period','final_period']].drop_duplicates()
    df_balanced['time_period'] = [time_periods for i in range(df_balanced.shape[0])]
    df_balanced = df_balanced.explode('time_period')
    df_balanced = df_balanced.query('time_period >= first_period & time_period <= final_period')
    df_contributor_panel_full = pd.merge(df_balanced, df_contributor_panel, how = 'left')

    return df_contributor_panel_full.drop(['first_period','final_period'], axis = 1).sort_values(['repo_name','actor_id','time_period'])

In [21]:
agg_cols = ['problem_identification', 'problem_discussion', 'coding', 'problem_approval']
df_contributor_panel_share = df_contributor_panel_rank[['actor_id', 'repo_name', 'time_period', 'max_rank'] + [f'{c}_share' for c in agg_cols]]

merged_dfs = [
    df_contributor_email_category,
    df_truckfactor_dates,
    df_contributor_panel_share,
    df_comments_grouped,
    df_pr_commits_grouped,
    df_pr_commits_100_grouped
]

df_contributor_characteristics = reduce(lambda left, right: left.merge(right, how='outer'), merged_dfs)
df_contributor_characteristics = MakeBalancedContributor(df_contributor_characteristics)
df_contributor_characteristics[['contributor_email_educational', 'contributor_email_corporate', 'truckfactor_member']] = df_contributor_characteristics[['contributor_email_educational', 'contributor_email_corporate', 'truckfactor_member']].fillna(0)

In [22]:
df_contributor_characteristics = df_contributor_characteristics.query('time_period>="2015-01-01"')

## Organizational Structure

**Layer Count**: Classify people into (at least) one of the four layers below, and for each layer, in that repo time-period, consider the quantity of contributors in that layer and the percentage of contributors who participated in that layer.  

**List of layers**
  - **Problem identification (Layer 1)**: Opened issue
  - **Problem discussion (Layer 2)**: Issue Comments + PR Comments + PR Review Comments
  - **Code writing (Layer 3)**: Commits in pushes, linked PRs or unlinked PRs (only include merged PRs)
  - **Problem approval (Layer 4)**: Closed issue + merged PR

**Min Layers**: Find the smallest subset of layers such that all contributors belong to at least one of the layers in the subset. All layers in the smallest subset are separated by "|" and if there are multiple smallest subsets, they are separated by ":"


In [23]:
LAYERS = agg_cols
def find_all_min_covers_for_group(grp):
    """
    Given the subset of the DataFrame for one (repo_name, time_period),
    return ALL minimal subsets of LAYERS that cover every actor in this group.

    Cover criterion:
      For each row (actor), at least one column in the chosen subset is True.
    """
    # We'll just slice the boolean columns
    subset_matrix = grp[LAYERS]

    # We'll track all minimal subsets found
    # If we never find any, it returns an empty list (though that likely won't happen
    # if there's at least one True in each group).
    min_solutions = []

    # 1) We try subsets of size 1, then size 2, etc., up to size 4
    # 2) Once we find ANY solutions for a given size r, we stop (those are minimal).
    for r in range(1, len(LAYERS) + 1):
        possible_combos = itertools.combinations(LAYERS, r)
        found_any = False

        for combo in possible_combos:
            # If every row is covered by 'combo', we keep it
            # "covered" means row-wise .any(axis=1) is True for all rows
            if subset_matrix[list(combo)].any(axis=1).all():
                min_solutions.append(combo)
                found_any = True

        if found_any:
            # We found at least one solution of size r, so there's no need
            # to look at larger subsets.
            break

    return pd.Series({'all_min_layers': min_solutions})

df_panel_min_layers = df_contributor_panel_rank\
    .groupby(['repo_name', 'time_period'], as_index=False)\
    .parallel_apply(find_all_min_covers_for_group)
df_panel_min_layers['min_layer_count'] = df_panel_min_layers['all_min_layers'].apply(lambda x: len(x[0]))
df_panel_min_layers['all_min_layers'] = df_panel_min_layers['all_min_layers'].apply(lambda x: ["|".join(sorted(list(ele))) for ele in x])

In [24]:
def GetPanelRankCount(df_contributor_panel_rank, agg_cols):
    df_panel_rank_count = df_contributor_panel_rank.groupby(['repo_name','time_period'])[agg_cols].sum()
    df_panel_rank_count.columns = [f"{col}_layer_contributor_count" for col in df_panel_rank_count.columns]
    
    df_panel_rank_pct = df_contributor_panel_rank.groupby(['repo_name','time_period'])[agg_cols].mean()
    df_panel_rank_pct.columns = [f"{col}_layer_contributor_pct" for col in df_panel_rank_pct.columns]
    
    df_panel_rank_count_pct = df_panel_rank_pct.join(df_panel_rank_count, how = 'outer').reset_index()
    df_panel_rank = df_panel_rank_pct.join(df_panel_rank_count, how = 'outer').reset_index()
    
    df_panel_layer_count = (df_panel_rank_count>0).sum(axis = 1).reset_index().rename({0:'layer_count'}, axis = 1)
    df_panel_rank = pd.merge(df_panel_rank, df_panel_layer_count)
    
    return df_panel_rank

df_panel_rank = GetPanelRankCount(df_contributor_panel_rank, agg_cols)
df_panel_rank_layers = pd.merge(df_panel_rank,df_panel_min_layers, how = 'outer')
df_panel_rank_layers['all_min_layers'] = df_panel_rank_layers['all_min_layers'].apply(lambda x: ":".join(x))

**Layer Overlap**
  1. **Overlap Contributor Pct**: For each layer $i \leq 4$ and layer $i < j \leq 4$, what % of individuals are in layer $i$ and $j$. Mathematically, this is $\frac{|\{P \in L_i \cap L_j\}|}{|\{P \in L_i\}|}$.
  2. **Degree of overlap**: For each layer $i \leq 4$ and layer $i < j \leq 4$, what % of work in the lower layer $i$ is done by individuals who are also in layer $j$: $\frac{\sum_{P \in L_i \cap L_j} W_P}{\sum_{P \in L_i} W_P}$
 
 The combinations are:
 - Problem Identification-Problem Discussion
 - Problem Identification-Coding
 - Problem Identification-Problem Approval
 - Problem Discussion-Coding
 - Problem Discussion-Problem Approval
 - Coding-Problem Approval
 
 I also calculate, for the degree of overlap, the % of work done by people who are in any higher layer.


In [25]:

def CalculateContributorOverlapTwoLayers(df_contributor_panel_rank, agg_cols):
    df_repo_panel_overlap = df_contributor_panel_rank[['repo_name', 'time_period']].drop_duplicates().reset_index(drop=True)
    for lower_layer, higher_layer in itertools.combinations(agg_cols, 2):
        overlap = (
            df_contributor_panel_rank
            .query(f'({lower_layer} == True)')
            .groupby(['repo_name', 'time_period'])[higher_layer]
            .mean()
            .reset_index()
            .rename(columns={higher_layer: f"{lower_layer}_{higher_layer}_contributor_overlap"})
        )
        df_repo_panel_overlap = pd.merge(df_repo_panel_overlap, overlap, on=['repo_name', 'time_period'], how='left')
        lower_higher_numerator = (
            df_contributor_panel_rank
            .query(f'({lower_layer} == True) & ({higher_layer} == True)')
            .groupby(['repo_name', 'time_period'])[f"{lower_layer}_sum"]
            .sum()
            .reset_index()
            .rename(columns={f"{lower_layer}_sum": f"{lower_layer}_{higher_layer}_work_numerator"})
        )
        lower_higher_denominator = (
            df_contributor_panel_rank
            .query(f'({lower_layer} == True)')
            .groupby(['repo_name', 'time_period'])[f"{lower_layer}_sum"]
            .sum()
            .reset_index()
            .rename(columns={f"{lower_layer}_sum": f"{lower_layer}_{higher_layer}_work_denominator"})
        )
        work_done = pd.merge(
            lower_higher_numerator, 
            lower_higher_denominator, 
            on=['repo_name', 'time_period'], 
            how='right'  # Ensure all groups with lower_layer=True are included
        )
        work_done[f"{lower_layer}_{higher_layer}_work"] = work_done.apply(
            lambda row: (
                row[f"{lower_layer}_{higher_layer}_work_numerator"] / 
                row[f"{lower_layer}_{higher_layer}_work_denominator"]
            ) if row[f"{lower_layer}_{higher_layer}_work_denominator"] > 0 and row[f"{lower_layer}_{higher_layer}_work_numerator"] > 0 else np.nan,
            axis=1
        )
        work_done = work_done[['repo_name', 'time_period', f"{lower_layer}_{higher_layer}_work"]]
        df_repo_panel_overlap = pd.merge(df_repo_panel_overlap, work_done, on=['repo_name', 'time_period'], how='left')

    return df_repo_panel_overlap

def CalculateContributorOverlap(df_contributor_panel_rank, agg_cols):
    df_repo_panel_overlap = df_contributor_panel_rank[['repo_name', 'time_period']].drop_duplicates().reset_index(drop=True)
    for i in range(len(agg_cols))[:-1]:
        lower_layer = agg_cols[i]
        higher_layer = agg_cols[(i+1):]
        higher_layer_query = " | ".join([f"{col} == True" for col in higher_layer])

        overlap = (
            df_contributor_panel_rank.assign(higher_layer = df_contributor_panel_rank[higher_layer].max(axis = 1))
            .query(f'({lower_layer} == True)')
            .groupby(['repo_name', 'time_period'])['higher_layer']
            .mean()
            .reset_index()
            .rename(columns={'higher_layer': f"{lower_layer}_higher_layer_contributor_overlap"})
        )
        df_repo_panel_overlap = pd.merge(df_repo_panel_overlap, overlap, on=['repo_name', 'time_period'], how='left')

        lower_higher_numerator = (
            df_contributor_panel_rank
            .query(f'({lower_layer} == True) & ({higher_layer_query})')
            .groupby(['repo_name', 'time_period'])[f"{lower_layer}_sum"]
            .sum()
            .reset_index()
            .rename(columns={f"{lower_layer}_sum": f"{lower_layer}_higher_layer_work_numerator"})
        )
        lower_higher_denominator = (
            df_contributor_panel_rank
            .query(f'({lower_layer} == True)')
            .groupby(['repo_name', 'time_period'])[f"{lower_layer}_sum"]
            .sum()
            .reset_index()
            .rename(columns={f"{lower_layer}_sum": f"{lower_layer}_higher_layer_work_denominator"})
        )
        work_done = pd.merge(
            lower_higher_numerator, 
            lower_higher_denominator, 
            on=['repo_name', 'time_period'], 
            how='right'  # Ensure all groups with lower_layer=True are included
        )
        work_done[f"{lower_layer}_higher_layer_work"] = work_done.apply(
            lambda row: (
                row[f"{lower_layer}_higher_layer_work_numerator"] / 
                row[f"{lower_layer}_higher_layer_work_denominator"]
            ) if row[f"{lower_layer}_higher_layer_work_denominator"] > 0 and row[f"{lower_layer}_higher_layer_work_numerator"] > 0 else np.nan,
            axis=1
        )
        work_done = work_done[['repo_name', 'time_period', f"{lower_layer}_higher_layer_work"]]
        df_repo_panel_overlap = pd.merge(df_repo_panel_overlap, work_done, on=['repo_name', 'time_period'], how='left')

    return df_repo_panel_overlap


df_repo_panel_overlap = CalculateContributorOverlap(df_contributor_panel_rank, agg_cols)

**Layer HHI (Work Split)**
  1. At the project-time period level, for all contributors $P$ who are involved in any layer $l_i$, $\sum_{P \in L_i} \left(S_{P, L_i} \right)^2$ where $S_{P,L_i}$ is the share of work that contributor $P$ did in layer $L_i$.
  2. <span style="color:red">TO DO: For each file $f$, using the git blame to determine the owner
     1. HHI (by overall LOC across all files)
     2. HHI (for each file, then average across all)</span>


In [26]:
def CalculateLayerHHI(df_contributor_panel_rank, agg_cols):
    df_repo_hhi = df_contributor_panel_rank[['repo_name', 'time_period']].drop_duplicates().reset_index(drop=True)

    for layer in agg_cols + ['total']:
        share_col = f"{layer}_share"
        hhi_col = f"{layer}_HHI"
        if layer != 'total':
            df_contributor = df_contributor_panel_rank.query(f"{layer} == True")
        else:
            df_contributor = df_contributor_panel_rank
        
        hhi = (
            df_contributor
            .assign(share_sq = df_contributor_panel_rank[share_col]**2)
            .groupby(['repo_name', 'time_period'])['share_sq']
            .sum()
            .reset_index()
            .rename(columns={'share_sq': hhi_col})
        )
        
        df_repo_hhi = pd.merge(df_repo_hhi, hhi, on=['repo_name', 'time_period'], how='left')
    

    
    return df_repo_hhi
df_repo_hhi = CalculateLayerHHI(df_contributor_panel_rank, agg_cols)

**Cooperation**:
1. For each discussion (includes comments, PR reviews + PR review comments) + PR, I calculate, across all threads in a project-time period
     1. **HHI:** $\frac{1}{|T_p|} \sum_{t \in T_p}^n HHI_{t}$ where $T_i$ contains all threads $t$ (discussions or pull requests) in a project-time period and $HHI_{t}$ represents the HHI of thread t
     2. **HHI, average weighted by discussion quantity:** $\sum_{t \in T_p}^n HHI_{t} \cdot \frac{C_{t}}{\sum_{t \in T_p} C_{t}}$ where $C_{t}$ represents the quantity of discussion done in thread t. Now, instead of weighting each problem equally, we weight each thread $t$ by how much work discussion occurred 
     3. **Average (across threads):** \# of distinct individuals per thread
     4. **Average (across threads, weighted by discussion quantity):** \# of distinct individuals per thread, weighted by the same quantity as HHI, average weighted by discussion quantity
     5. **Cooperation Occurred:** \% of threads with more than one distinct individual


In [27]:
def calculate_hhi_cooperation(df, entity, metric):
    metric_share = f"{metric}_share"               
    entity_number = f"{entity}_number"             
    metric_col = metric                            
    total_metric = f"total_{metric}"               
    repo_metric = f"repo_{metric}"                 
    contributors_metric = f"contributors_{metric}"
    contributors_metric_wt = f"{contributors_metric}_wt"
    
    hhi_col = f"{entity}_hhi_{metric}"             
    cooperation_col = f"{metric}_cooperation"      
    wt_work_done_col = f"{entity}_{metric}_wt"
    
    df[hhi_col] = df.assign(share_sq = df[metric_share] ** 2)\
        .groupby(['repo_name', 'time_period', entity_number])['share_sq'].transform('sum')
    
    df[cooperation_col] = df[hhi_col] != 1
    df[repo_metric] = df.groupby(['repo_name', 'time_period'])[metric_col].transform('sum')
    
    df[wt_work_done_col] = df[hhi_col] * df[total_metric] / df[repo_metric]
    
    df[contributors_metric] = df.groupby(['repo_name', 'time_period', entity_number])[metric_share].transform('count')
    df[contributors_metric_wt] = df[contributors_metric] * df[total_metric] / df[repo_metric]
    
    df_unique = df.drop_duplicates(['repo_name', 'time_period', entity_number])
    
    df_cooperation = df_unique.groupby(['repo_name', 'time_period']).agg(
        hhi_metric = (hhi_col, 'mean'),
        hhi_metric_wt = (wt_work_done_col, 'sum'),
        contributors_metric = (contributors_metric, 'mean'),
        contributors_metric_wt = (contributors_metric_wt, 'sum'),
        metric_cooperation_pct = (cooperation_col, 'mean')
    ).reset_index()
    
    df_cooperation = df_cooperation.rename(columns={
        'hhi_metric': f"hhi_{metric}",
        'hhi_metric_wt': f"hhi_{metric}_wt",
        'contributors_metric':f"contributors_{metric}",
        'contributors_metric_wt':f"contributors_{metric}_wt",
        'metric_cooperation_pct': f"{metric}_cooperation_pct"
    })
    
    
    return df_cooperation


In [28]:
df_comments_cooperation = calculate_hhi_cooperation(df_comments_actor, 'issue','comments')
df_commits_cooperation = calculate_hhi_cooperation(df_pr_commits_actor, 'pr','commits_count')

**communication**
- Given a contributor's rank, what % of discussion occurs with people whose rank is (1), (2), (3), (4)
  - get 1/x weight if they are in x ranks (for the % of discussion)
  - Average weighted by contributions of person, or by person
- Can aggregate for each contributor into a weighted average of the average rank they communicate with 

In [29]:
df_comments_actor_rank = pd.merge(df_comments_actor, df_contributor_panel_rank[['actor_id','repo_name','time_period','max_rank'] + agg_cols].drop_duplicates(), how = 'left')

In [30]:
def RankParticipationFlags(df, entity='issue'):
    entity_number = f"{entity}_number"

    ranks = [1, 2, 3, 4]

    rank_cols = [f'rank_{rank}' for rank in ranks]
    for rank in ranks:
        df[f'rank_{rank}'] = (df['max_rank'] == rank).astype(int)
    df['num_layers'] = df[rank_cols].sum()
    
    group_cols = ['repo_name', 'time_period', entity_number]
    rank_presence = df.groupby(group_cols)[rank_cols].max().reset_index()

    
    participation_cols = [f'participated_rank_{rank}' for rank in ranks]
    rename_dict = {f'rank_{rank}': f'participated_rank_{rank}' for rank in ranks}
    rank_presence = rank_presence.rename(columns=rename_dict)

    df = df.merge(rank_presence, on=group_cols, how='left')
    df = df.drop(columns=rank_cols)

    return df

def calculate_rank_metrics(df, entity_number='issue_number', metric='comments', ranks=[1,2,3,4],
                           weight_column='comments_discussions_involved'):

    df = RankParticipationFlags(df)

    # issue by issue average
    df_wt_by_issue = df.drop_duplicates(['repo_name','time_period',entity_number,'max_rank']).groupby(['repo_name','time_period','max_rank']).agg(
        participated_rank_1 = ('participated_rank_1','mean'),
        participated_rank_2 = ('participated_rank_2','mean'),
        participated_rank_3 = ('participated_rank_3','mean'),
        participated_rank_4 = ('participated_rank_4','mean'),
        discussion_count = (entity_number,'nunique')
    )

    df_rank_count = df.drop_duplicates(['repo_name','time_period','max_rank','actor_id']).groupby(
        ['repo_name','time_period','max_rank'])['actor_id'].count().rename('rank_contributor_count')
    
    df_rank_summary = df_wt_by_issue.join(df_rank_count)
    
    return df_rank_summary.reset_index()

def MakeWide(df_comments_rank_summary):
    df_wide = df_comments_rank_summary.sort_values('max_rank').pivot_table(
        index=['repo_name', 'time_period'],
        columns='max_rank',
        aggfunc='first'
    )
    df_wide.columns = [f"max_rank_{int(rank)}_{col}" for col, rank in df_wide.columns]
    df_wide.reset_index(inplace=True)
    return df_wide


In [31]:
#df_comments_rank_summary = calculate_rank_metrics(df_comments_actor_rank)
#df_comments_rank_summary_wide = MakeWide(df_comments_rank_summary)
"""rank_list = df_contributor_panel_rank.groupby(['repo_name','time_period']).agg({'max_rank':lambda x: sorted(set(x))}).rename({'max_rank':'rank_list'}, axis = 1).reset_index()
df_rank_wide_cooperation = pd.merge(df_comments_rank_summary_wide, rank_list, how = 'left')

df_comments_rank_summary_wide[['max_rank_2_rank_contributor_count','max_rank_3_rank_contributor_count','max_rank_4_rank_contributor_count']] = \
    df_comments_rank_summary_wide[['max_rank_2_rank_contributor_count','max_rank_3_rank_contributor_count','max_rank_4_rank_contributor_count']].fillna(0)

for rank in [2,3,4]:
    df_rank_wide_cooperation[f'max_rank{rank}_discussion_count'] = df_rank_wide_cooperation.apply(
        lambda x: x[f'max_rank{rank}_discussion_count'] if not pd.isnull(x[f'max_rank{rank}_discussion_count']) else
        0 if rank in x['rank_list'] else np.nan, axis = 1)
    

# max_rank_{rank}_discussion_count is 0 if rank is in rank_list
# max_rank_{rank}_{rank2} is 0 if rank is in rank_list & max_rank_{rank}_discussion_count>0 (not nan)
"""

"rank_list = df_contributor_panel_rank.groupby(['repo_name','time_period']).agg({'max_rank':lambda x: sorted(set(x))}).rename({'max_rank':'rank_list'}, axis = 1).reset_index()\ndf_rank_wide_cooperation = pd.merge(df_comments_rank_summary_wide, rank_list, how = 'left')\n\ndf_comments_rank_summary_wide[['max_rank_2_rank_contributor_count','max_rank_3_rank_contributor_count','max_rank_4_rank_contributor_count']] =     df_comments_rank_summary_wide[['max_rank_2_rank_contributor_count','max_rank_3_rank_contributor_count','max_rank_4_rank_contributor_count']].fillna(0)\n\nfor rank in [2,3,4]:\n    df_rank_wide_cooperation[f'max_rank{rank}_discussion_count'] = df_rank_wide_cooperation.apply(\n        lambda x: x[f'max_rank{rank}_discussion_count'] if not pd.isnull(x[f'max_rank{rank}_discussion_count']) else\n        0 if rank in x['rank_list'] else np.nan, axis = 1)\n    \n\n# max_rank_{rank}_discussion_count is 0 if rank is in rank_list\n# max_rank_{rank}_{rank2} is 0 if rank is in rank_list

## Combine Covariates

In [32]:
df_repo_panel_org_structure = df_panel_rank_layers.merge(df_repo_panel_overlap, how = 'outer').merge(df_repo_hhi, how = 'outer').merge(df_comments_cooperation, how = 'outer').merge(df_commits_cooperation, how = 'outer')
df_repo_panel_org_structure = MakeBalanced(df_repo_panel_org_structure)

In [33]:
df_repo_panel_org_structure = df_repo_panel_org_structure.query('time_period>="2015-01-01"')

# Combine All

In [34]:
df_repo_covariates = pd.merge(df_repo_characteristics, df_repo_panel_org_structure, how = 'outer')
df_repo_covariates = FillRepoCharacteristics(df_repo_covariates)
df_repo_covariates['org_status'] = df_repo_covariates['repo_name'].apply(lambda x: x.split("/")[0] in unique_orgs_list)
df_repo_covariates.to_parquet('issue/project_covariates.parquet')
df_contributor_characteristics.to_parquet('issue/contributor_covariates.parquet')